In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
import numpy as np

# --- 1. Load Data (CIFAR-10) ---
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define class names for reporting
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
NUM_CLASSES = len(class_names)
INPUT_SHAPE = x_train.shape[1:]  # (32, 32, 3)

print(f"Training samples: {x_train.shape[0]}, Image shape: {INPUT_SHAPE}")

# --- 2. Build Lightweight Model (Simple CNN) ---
model = models.Sequential([
    # Input layer and first Conv/Pool block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
    layers.MaxPooling2D((2, 2)),

    # Second Conv/Pool block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Classification head
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# --- 3. Compile and Train ---
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

EPOCHS = 10 # Increase for better accuracy, but longer training
history = model.fit(x_train, y_train, epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    verbose=1)

# --- 4. Evaluate and Save ---
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest Accuracy (Keras Model): {test_accuracy*100:.2f}%")

model.save('cifar10_cnn_classifier.h5')
print("\nKeras model saved as cifar10_cnn_classifier.h5")

Training samples: 50000, Image shape: (32, 32, 3)
Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 68s 42ms/step - accuracy: 0.3444 - loss: 1.7755 - val_accuracy: 0.5622 - val_loss: 1.2437
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 66s 42ms/step - accuracy: 0.5827 - loss: 1.1945 - val_accuracy: 0.6206 - val_loss: 1.0824
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 66s 42ms/step - accuracy: 0.6403 - loss: 1.0378 - val_accuracy: 0.6525 - val_loss: 1.0018
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 42ms/step - accuracy: 0.6707 - loss: 0.9508 - val_accuracy: 0.6658 - val_loss: 0.9711
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 42ms/step - accuracy: 0.6895 - loss: 0.8981 - val_accuracy: 0.6622 - val_loss: 0.9857
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 66s 42ms/step - accuracy: 0.7064 - loss: 0.8507 - val_accuracy: 0.6804 - val_loss: 0.9352
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 42ms/step - accuracy: 0.7185 - loss: 0.8180 - val_accuracy: 0.6883 - val_loss: 0.9195
Epoch 8/10
1563/1563 ━━━━━


Test Accuracy (Keras Model): 69.18%

Keras model saved as cifar10_cnn_classifier.h5


In [ ]:
# --- 5. Convert Keras Model to TFLite ---
TFLITE_FILE_PATH = 'cifar10_cnn_classifier.tflite'

# Load the saved Keras model
keras_model = tf.keras.models.load_model('cifar10_cnn_classifier.h5')

# Convert the Keras model to a TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Save the TFLite model
with open(TFLITE_FILE_PATH, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved as {TFLITE_FILE_PATH}")

# --- 6. Load and Test TFLite Model ---
interpreter = tf.lite.Interpreter(model_path=TFLITE_FILE_PATH)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test on a subset of the test data (e.g., the first 1000 samples)
num_samples = 1000
x_sample = x_test[:num_samples]
y_true = y_test[:num_samples].flatten()
tflite_predictions = []

# TFLite input requires specific type (usually float32)
input_dtype = input_details[0]['dtype']

for i in range(num_samples):
    input_data = x_sample[i:i+1].astype(input_dtype)

    # Set the tensor, invoke, and get output
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    tflite_output = interpreter.get_tensor(output_details[0]['index'])

    predicted_label = np.argmax(tflite_output[0])
    tflite_predictions.append(predicted_label)

tflite_accuracy = np.mean(tflite_predictions == y_true)
print(f"\n**TFLite Model Accuracy (Sample Test): {tflite_accuracy*100:.2f}%**")

Saved artifact at '/tmp/tmpv04y2xgs'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  136704935370320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136704935370128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136704935370896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136704935371280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136704935372048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136704935370512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136704935377808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136704935371856: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model saved as cifar10_cnn_classifier.tflite


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)



**TFLite Model Accuracy (Sample Test): 71.40%**
